In [40]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('Algorithm.py'))))
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('Environment.py'))))
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('Replay_Buffer.py'))))
import Algorithm as ALGS
import Environment as ENVS
import Replay_Buffer


In [41]:
# import collections

# def initialize_buffer(config):
#     return SimpleExperienceBuffer(config['max_size'], config['batch_size'])

# class SimpleExperienceBuffer:
#     def __init__(self, capacity, batch_size):
#         self.buffer = collections.deque(maxlen=capacity)
#         self.batch_size = batch_size

#     def __len__(self):
#         return len(self.buffer)

#     def append(self, experience):
#         self.buffer.append(experience)

#     def sample(self):
#         indices = np.random.choice(len(self.buffer), self.batch_size)
#         states, actions, rewards, dones, next_states = zip(*[self.buffer[idx] for idx in indices])
#         return np.array(states), np.array(actions), np.array(rewards, dtype=np.float32), \
#                np.array(dones, dtype=np.uint8), np.array(next_states)

# Implementation Example

## First initialize stuff

In [42]:
config = {'Learner': {'type': 'DQN', 'episodes': 1}, 'Algorithm': {'algorithm': 'DQN', 'replay_buffer': True, 'learning_rate': 0.003, 'optimizer': 'Adam', 'loss_function': 'MSELoss', 'regularizer': 0, 'recurrence': 0, 'gamma': 0.99, 'beta': 0, 'epsilon_start': 1, 'epsilon_end': 0.02, 'epsilon_decay': 5e-05, 'c': 10000}, 'Environment': {'env_type': 'Gym', 'environment': 'CartPole-v0', 'action_space': 'discrete', 'observation_space': 'discrete', 'env_render': False, 'num_agents': 1}, 'Replay_Buffer': {'max_size': 100000, 'batch_size': 64, 'num_agents': 1}, 'Agent': {'num_agents': 1}, 'Network': {'algorithm': 'DQN', 'layers': 3, 'hidden_layer1': 'linear', 'hidden_size1': 50, 'activation_function1': 'relu', 'hidden_layer2': 'linear', 'hidden_size2': 100, 'activation_function2': 'relu', 'hidden_layer3': 'linear'}}

In [43]:
config

{'Learner': {'type': 'DQN', 'episodes': 1},
 'Algorithm': {'algorithm': 'DQN',
  'replay_buffer': True,
  'learning_rate': 0.003,
  'optimizer': 'Adam',
  'loss_function': 'MSELoss',
  'regularizer': 0,
  'recurrence': 0,
  'gamma': 0.99,
  'beta': 0,
  'epsilon_start': 1,
  'epsilon_end': 0.02,
  'epsilon_decay': 5e-05,
  'c': 10000},
 'Environment': {'env_type': 'Gym',
  'environment': 'CartPole-v0',
  'action_space': 'discrete',
  'observation_space': 'discrete',
  'env_render': False,
  'num_agents': 1},
 'Replay_Buffer': {'max_size': 100000, 'batch_size': 64, 'num_agents': 1},
 'Agent': {'num_agents': 1},
 'Network': {'algorithm': 'DQN',
  'layers': 3,
  'hidden_layer1': 'linear',
  'hidden_size1': 50,
  'activation_function1': 'relu',
  'hidden_layer2': 'linear',
  'hidden_size2': 100,
  'activation_function2': 'relu',
  'hidden_layer3': 'linear'}}

In [44]:
env = ENVS.initialize_env(config['Environment'])
env

In [45]:
alg = ALGS.initialize_algorithm(env.get_observation_space(), env.get_action_space(), [config['Algorithm'], config['Agent'], config['Network']])
alg

In [46]:
agent = alg.create_agent()
agent

FileNotFoundError: [Errno 2] No such file or directory: '/home/cabesd/Documents/nflux/Control-Tasks/Shiva/Testing/Shiva/DQNet1.py'

In [47]:
buffer = Replay_Buffer.initialize_buffer(config['Replay_Buffer'], None, None, None)
buffer

## Now simulate some training

In [48]:
num_of_episodes = 50

for i in range(num_of_episodes):
    obs = env.reset()
    obs = env.get_observation()
    done = False
    while not done:
        action = alg.get_action(agent, obs, i)
        next_obs, reward, done = env.step(action)
        
        experience = [obs, action, reward, done, next_obs]
        buffer.append(experience)
        
        experience = buffer.sample()
        alg.update(agent, experience, i)

        obs = next_obs

NameError: name 'agent' is not defined

In [49]:
obs, actions, rewards, dones, next_obs = buffer.sample()
obs.shape, actions.shape, rewards.shape, dones.shape, next_obs.shape

ValueError: a must be greater than 0 unless no samples are taken

In [147]:
def action2one_hot_v(action: int):
    z = torch.zeros(4)
    z[action] = 1
    return z

In [148]:
def action2one_hot(action: int):
    z = np.zeros(4)
    z[action] = 1
    return z

In [149]:
i = 1
act = action2one_hot(i)
type(act)

numpy.ndarray

In [150]:
act2 = action2one_hot_v(i)
act2

tensor([0., 1., 0., 0.])

In [154]:
states = [ np.random.rand(4) for i in range(4) ]
actions = [ action2one_hot(np.random.randint(4)) for i in range(4) ]

In [155]:
states

[array([0.73574718, 0.18711238, 0.944074  , 0.29415164]),
 array([0.50268395, 0.75168611, 0.12315211, 0.18701906]),
 array([0.96479454, 0.36139338, 0.82702615, 0.13096865]),
 array([0.59141063, 0.41393826, 0.1167971 , 0.75477607])]

In [156]:
actions

[array([0., 0., 1., 0.]),
 array([0., 0., 1., 0.]),
 array([0., 1., 0., 0.]),
 array([0., 1., 0., 0.])]

In [170]:
input_v = torch.tensor([ np.concatenate([s_i, a_i]) for s_i, a_i in zip(states, actions) ])
input_v

tensor([[0.7357, 0.1871, 0.9441, 0.2942, 0.0000, 0.0000, 1.0000, 0.0000],
        [0.5027, 0.7517, 0.1232, 0.1870, 0.0000, 0.0000, 1.0000, 0.0000],
        [0.9648, 0.3614, 0.8270, 0.1310, 0.0000, 1.0000, 0.0000, 0.0000],
        [0.5914, 0.4139, 0.1168, 0.7548, 0.0000, 1.0000, 0.0000, 0.0000]],
       dtype=torch.float64)

In [171]:
net = torch.nn.Sequential(
            nn.Linear(8, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
        )

In [176]:
state_action_val = net(input_v.float())

In [186]:
dones = [True, False, False, True]
done_mask = torch.ByteTensor(dones)#.to(device)
done_mask

tensor([1, 0, 0, 1], dtype=torch.uint8)

In [188]:
state_action_val[done_mask] = 0.00
state_action_val

/pytorch/aten/src/ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.


tensor([[ 0.0000],
        [-0.1573],
        [-0.1679],
        [ 0.0000]], grad_fn=<IndexPutBackward>)